In [1]:
import os
import sys
from pathlib import Path
from confluent_kafka import DeserializingConsumer
from confluent_kafka.serialization import StringDeserializer
from google.protobuf.message import DecodeError
from dotenv import load_dotenv

# Import the generated Protobuf schema
sys.path.append('../protobuf')
import schema_pb2

# Load environment variables from .env file
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

kafka_host = os.getenv('KAFKA_HOST')
topic_name = os.getenv('KAFKA_TOPIC_NAME')
group_id = os.getenv('CONSUMER_GROUP', 'petshop-consumer-group')

# Kafka Configuration
bootstrap_servers = f'{kafka_host}:9092'

# Consumer Configuration
consumer_conf = {
    'bootstrap.servers': bootstrap_servers,
    'key.deserializer': StringDeserializer('utf_8'),  # Deserializer for key
    'value.deserializer': lambda x: x,  # Raw bytes deserializer (Protobuf)
    'group.id': group_id,
    'auto.offset.reset': 'earliest'  # Start consuming from the earliest available message
}

# Initialize Kafka consumer
consumer = DeserializingConsumer(consumer_conf)

# Subscribe to the topic
consumer.subscribe([topic_name])

# Variables to perform calculations
total_price = 0
total_weight = 0
event_count = 0

def consume_and_calculate():
    """Consume Protobuf-encoded messages from Kafka and perform calculations."""
    global total_price, total_weight, event_count

    while True:
        try:
            # Poll for a message
            msg = consumer.poll(1.0)  # Timeout of 1 second

            if msg is None:
                continue  # No message received, continue polling

            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue

            # Deserialize the received message
            pet_event = schema_pb2.PetShopEvent()

            try:
                pet_event.ParseFromString(msg.value())
            except DecodeError as e:
                print(f"Error decoding Protobuf message: {e}")
                continue

            # Perform calculations on the consumed data
            total_price += pet_event.price
            total_weight += pet_event.weight
            event_count += 1

            # Calculate average weight
            average_weight = total_weight / event_count if event_count > 0 else 0

            # Print event and current calculations
            print(f"Consumed PetShopEvent: {pet_event}")
            print(f"Current Total Price: {total_price}")
            print(f"Current Average Weight: {average_weight}")
            print(f"Total Events Processed: {event_count}\n")

        except KeyboardInterrupt:
            break

    # Close the consumer after use
    consumer.close()

if __name__ == "__main__":
    consume_and_calculate()
